Pip installing some required dependencies

In [1]:
!pip install --upgrade pip
!pip install ffmpeg-python
!pip install moviepy

Requirement already up-to-date: pip in /opt/conda/lib/python3.6/site-packages (18.0)


Setting matplotlibs backend to agg so dask will not display, before it's imported

In [1]:
%matplotlib agg
#%matplotlib inline

In [2]:
import iris
import matplotlib.pyplot as plt
import matplotlib.cm as mpl_cm
import iris.quickplot as qplt
import iris.plot as iplt
from mpl_toolkits import mplot3d
import numpy as np
from iris.experimental.equalise_cubes import equalise_attributes
import iris.plot as iplot
import iris.analysis.cartography
from matplotlib.transforms import offset_copy
import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt
import os
from iris.cube import CubeList
from dask import delayed
import dask.bag as db
from distributed import Client
from matplotlib.widgets import Slider
import ffmpeg
import subprocess
import matplotlib.font_manager as fm
import matplotlib.colors as colors
from dask_kubernetes import KubeCluster
import pickle
import matplotlib

In [3]:
from moviepy.editor import *
import imageio

In [4]:
imageio.plugins.ffmpeg.download()

Getting filepaths in an appropriate format for plotting

In [5]:
def define_filepaths(filepath):
    filenames = !ls filepath
    filepaths = [os.path.join(filepath, file) for file in filenames]
    sequence = list(enumerate(filepaths))
    return sequence

In [ ]:
sequence = define_filepaths(__filepath__)

Function that makes an iris cube from the file and plots it, saving as png file

In [7]:
def get_map_plot(filetuple, save_filepath):
    filenumber, filename = filetuple
    
    cubelist = iris.load(filename)

    for c in cubelist:
        c.rename('v')
        n = int(c.attributes['Z_plus_Z_averaging'][7:10])
        c.add_aux_coord(iris.coords.AuxCoord(n, 'height'))
    
    equalise_attributes(cubelist)
    cubelist = cubelist.merge_cube()
    cubelist.data = np.ma.masked_where(cubelist.data <= 0.00001, cubelist.data)
    cubelist = cubelist.collapsed('height', iris.analysis.SUM)
    subset = cubelist[0]#.intersection(longitude=(-18,110), latitude=(0,50))
    subset.convert_units('microgram/m3')
    
    plt.switch_backend('agg')
    
    MetOfficeFont = fm.FontProperties(fname='/scratch/liz/FSEmeric-Regular.ttf')
    MetOfficeFontBold = fm.FontProperties(fname='/scratch/liz/FSEmeric-Heavy.ttf')
    
    fig, ax = plt.subplots(figsize=(9.6,5.4))
    
    stamen_terrain = cimgt.StamenTerrain()
    fig = plt.axes(projection=stamen_terrain.crs)
    fig.add_image(stamen_terrain, 4)
    #plt.title('Simulation of sulphur dioxide concentration after 2011 Nabro volcano eruption',  fontproperties=MetOfficeFontBold)
    contour_result = iplt.pcolormesh(subset, norm=colors.LogNorm(vmin=9.8203514, vmax=24652.268), cmap=mpl_cm.get_cmap('magma'))
    plt.gca().coastlines('50m')
    
    cbaxes = plt.axes([0.2, 0.25, 0.65, 0.03])
    ourcolorbar = plt.colorbar(contour_result, cax=cbaxes, orientation= 'horizontal')
    #ourcolorbar.set_ticks([10, 100, 1000, 10000])
    #ourcolorbar.set_ticklabels([10, 100, 1000, 10000])
    
    ourcolorbar.set_label('Sulphur dioxide air concentration (Micrograms/Metre³)', fontproperties=MetOfficeFont, color='#FFFFFF', fontsize=12, bbox=dict(facecolor='#007AA9', edgecolor='#2A2A2A', boxstyle='square'))
    
    fig.plot(41.7, 13.366667, marker='^', color='#B9DC0C', markersize=7, transform=ccrs.Geodetic())
    geodetic_transform = ccrs.Geodetic()._as_mpl_transform(fig)
    text_transform = offset_copy(geodetic_transform, units='dots', y=+75)
    fig.text(41.7, 13.366667, u'Nabro', fontproperties=MetOfficeFont, alpha=0.7, fontsize=12,verticalalignment='center', horizontalalignment='right',transform=text_transform,bbox=dict(facecolor='#B9DC0C', edgecolor='#2A2A2A', boxstyle='round'))
    
    attributedict = subset.attributes
    datetime = attributedict.get('time')
    timetransform = offset_copy(geodetic_transform, units='dots', y=0)
    fig.text(60, 0, "Time, date: "+ datetime[:2] + ':' + datetime[2:], fontproperties=MetOfficeFont, alpha=0.7, fontsize=12 , transform=timetransform, bbox=dict(facecolor='#B9DC0C', edgecolor='#2A2A2A', boxstyle='round'))
    
    titleaxes = plt.axes([0.2, 0.8, 0.65, 0.04], facecolor='#007AA9')
    titleaxes.text(0.5,0.25,'Simulation of sulphur dioxide concentration after 2011 Nabro volcano eruption', horizontalalignment = 'center',  fontproperties=MetOfficeFontBold, color = '#FFFFFF')
    titleaxes.set_yticks([])
    titleaxes.set_xticks([])

        
    #picturename = "%04i.png" % filenumber
    picturename = save_filepath + "%04i.png" % filenumber
    plt.savefig(picturename, dpi=200, frameon=False, bbox_inches='tight')
    #plt.show()

Defining dask scheduler

In [8]:
#cluster = KubeCluster.from_yaml('/temp/worker-template.yaml')
cluster = KubeCluster()

In [9]:
cluster

In [10]:
_ = cluster.scale(10)

In [10]:
client = Client(cluster.scheduler_address)

In [11]:
client

Client Scheduler: tcp://100.97.109.20:42745 Dashboard: http://100.97.109.20:8787/status,Cluster Workers: 6 Cores: 6 Memory: 36.00 GB


Running plots function on all filepaths with dask

In [14]:
make_plots = db.from_sequence(sequence).map(get_map_plot)
plotting = make_plots.compute()

Making video with ffmpeg

In [15]:
def make_video(end_vid, save_filepath):
    return_value = subprocess.call(["ffmpeg", "-i", save_filepath, "-vcodec", "mpeg4", end_vid])
    return return_value

make_video(__end_vid__, __save_filepath__)

0

Some more options we didn't need in this video - resizing it's bytes, and interpolating the frames.

In [ ]:
clip = mp.VideoFileClip("volcano_test_vid_minterpolate.mp4")
clip_resized = clip.resize(height=1000)
clip_resized.write_videofile("volcano_test_new.mp4")

In [ ]:
def smooth_existing_video(video):
    return_value = subprocess.call(["/home/jovyan/.imageio/ffmpeg/ffmpeg-linux64-v3.3.1", "-i", video, "-filter", "minterpolate", "really_hopefully_minterpolated.mp4"])
    return return_value

smooth_existing_video("/home/jovyan/Summer-projects/volcano_test_vid_minterpolate.mp4")